# 第8章 类与对象
## 8.1 修改实例的字符串表示
### 8.1.1 问题
修改打印实例所产生的输出，使输出结果能更有意义。
### 8.1.2 解决方案
通过定义`__str__()`和`__repr__()`来实现

In [1]:
class Pair:
    def __init__(self,x,y):
        self.x = x
        self.y = y
    
    def __repr__(self):
        return 'Pair({},{})'.format(self.x,self.y)

    def __str__(self):
        return '({},{})'.format(self.x,self.y)

In [2]:
p = Pair(3,4)
p # __repr__() 输出

Pair(3,4)

In [3]:
print(p) # __str__() 输出

(3,4)


In [4]:
# 在进行格式化输出时，特殊的格式化代码!r 表示应该使用__repr__()的输出
print('p is {0!r}'.format(p))
print('p is {0}'.format(p))

p is Pair(3,4)
p is (3,4)


## 8.2 自定义字符串的输出格式
### 8.2.1 问题
我们想让对象通过 format()函数和字符串方法来支持自定义的输出格式。
### 8.2.2 解决方案
要自定义字符串的输出格式，可以在类中定义`__format__()`方法。

In [8]:
_formats = {
    'ymd': '{d.year}-{d.month}-{d.day}',
    'mdy': '{d.month}/{d.day}/{d.year}',
    'dmy': '{d.day}/{d.month}/{d.year}'
}
class Date:
    def __init__(self,year,month,day):
        self.year = year
        self.month = month
        self.day = day
    
    def __format__(self,code) -> str:
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d = self)

d = Date(year=2023,month=9,day=4)
format(d),format(d,'mdy')

('2023-9-4', '9/4/2023')

In [9]:
'The date is {:ymd}'.format(d),'The date is {:mdy}'.format(d)

('The date is 2023-9-4', 'The date is 9/4/2023')

## 8.3 让对象支持上下文管理协议
### 8.3.1 问题
我们想让对象支持上下文管理协议（通过with语句触发）
### 8.3.2 解决方案
要让对象能够兼容with语句，与需要实现`__enter__()`和`__exit__()`方法。

In [11]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address ,family = AF_INET, type = SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.connections = []
    
    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

conn = LazyConnection(('www.baidu.com', 80))
with conn as s1:
    print('连接成功')
    print(s1)


连接成功
<socket.socket fd=1360, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.177.77', 52602), raddr=('180.101.50.242', 80)>


## 8.4 当创建大量实例时如何节省内存
### 8.4.1 问题
我们的程序创建了大量的（比如百万级）实例，为此占用了大量的内存
### 8.4.2 解决方案
对于那些主要用作简单数据结构的类，通常可以在类定义中增加`__slot__`属性，以此来大量减少对内存的使用。

In [12]:
# 我们被限制为只允许使用__slots__中列出的那些属性名
class Date:
    __slot__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

## 8.5 将名称封装到类中
### 8.5.1 问题
我们想将“私有”数据封装到类的实例上，但是又需要考虑到python缺乏对属性的访问控制问题
### 8.5.2 解决方案
与其依赖语言特性来封装数据，Python 程序员们更期望通过特定的命名规则来表达出对数据和方法的用途。第一个规则是任何以单下划线（_）开头的名字应该总是被认为只属于内部实现。

In [1]:
class A:
    def __init__(self):
        self._internal = 0 # An internal attribute
        self.public = 1 # An internal attribute
    
    def  public_method(self):
        """
        A public method
        """
        pass

    def _internal_method(self):
        """
        """
        pass
    

In [14]:
class B:
    def __init__(self):
        self.__private = 0
    def __private_method(self):
        ...
    def public_method(self):
        ...
        self.__private_method()
        ...
# 以双下划线打头的名称会导致出现名称重整（name mangling）的行为
# 这里 ，私有名称 __private 和__private_method 会被重 命名为_C__private 和 _C__
# private_method，这和基类 B 中的重整名称不同
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B.__private
# Does not override B.__private_method()
    def __private_method(self):
        ...

ass = C()
ass._B__private, ass._C__private

(0, 1)

## 8.6 创建可管理的属性
### 8.6.1 问题
在对实例属性的获取和设定上，我们希望增加一些额外的处理过程（比如类型检查或者验证）。
### 8.6.2 解决方案
要自定义对属性的访问，一种简单的方式是将其定义为 property。

In [3]:
class Person:
    def __init__(self,first_name):
        self.first_name = first_name
    
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value,str):
            raise TypeError("请传入字符串")
        self._first_name = value
    
    @first_name.deleter
    def first_name(self):
        raise AttributeError("不能删除")

a = Person('guide')
print(a.first_name)

guide


In [16]:
a.first_name = 12

TypeError: 请传入字符串

In [17]:
del a.first_name

AttributeError: 不能删除

## 8.7 调用父类中的方法
### 8.7.1 问题
我们想调用一个父类中的方法，这个方法在子类中已经被覆盖了。
### 8.7.2 解决方案
要调用父类（或称超类）中的方法，可以使用 super()函数完成。

In [2]:
class A:
    def spam(self):
        print('A.spam')

class B(A):
    def spam(self):
        print('B.spam')
        super().spam()

b = B()
b.spam()

B.spam
A.spam


## 8.8 在子类中扩展属性
### 8.8.1 问题
我们想在子类中扩展某个属性的功能，而这个属性是在父类中定义的。
### 8.8.2 解决方案

In [10]:
# 下面从 Person 类中继承，然后在子类中扩展 name 属性的功能
class Person:
    def __init__(self,name):
        self.name = name
    
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        if not isinstance(value,str):
            raise TypeError("请传入字符串")
        self._name = value
    
    @name.deleter
    def name(self):
        raise AttributeError("不能删除")
    
class SubPerson(Person):
    @property
    def name(self):
        print('getting name ')
        return super().name
    
    @name.setter
    def name(self,value):
        print('setting name to ', value)
        super(SubPerson, SubPerson).name.__set__(self,value)
    
    @name.deleter
    def name(self):
        print('deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

s = SubPerson('Guido')
print(s.name)
s.name = 'xiny'
s.name = 12

setting name to  Guido
getting name 
Guido
setting name to  xiny
setting name to  12


TypeError: 请传入字符串

In [12]:
# 只想扩展属性中的其中一个方法
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('getting name')
        return super().name

a = SubPerson('abc')
a.name

getting name


'abc'

## 8.9 创建一种新形式的类属性或实例属性
### 8.9.1 问题
我们想创建一种新形式的实例属性，它可以拥有一些额外的功能，比如说类型检查。
### 8.9.2 解决方案


In [14]:
class Integer:
    def __init__(self,name):
        self.name = name
    
    def __get__(self,instance,cls):
        if instance is None: 
            return self
        else:
            return instance.__dict__[self.name]
    
    def __set__(self,instance,value):
        if not isinstance(value,int):
            raise TypeError('应输入int')
        instance.__dict__[self.name] = value
    
    def __delete__(self,instance):
        del instance.__dict__[self.name]

class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self,x,y):
        self.x = x
        self.y = y

p = Point(2,3)
print(p.x)
p.y = 5
p.x = 2.3

2


TypeError: 应输入int

## 8.10 让属性具有惰性求值的能力
### 8.10.1 问题
我们想将一个只读的属性定义为property属性方法，只有在访问它时才参与计算。但是，一旦访问了该属性，我们希望把计算出的值缓存起来，不要每次访问它时都重新计算
### 8.10.2 解决方案

In [15]:
class lazyproperty:
    def __init__(self,func):
        self.func = func
    
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance,self.func.__name__,value)
            return value

import math
class Circle:
    def __init__(self,radius):
        self.radius = radius

    @lazyproperty
    def area(self):
        print('Computer area')
        return math.pi * self.radius ** 2
    
    @lazyproperty
    def perimeter(self):
        print('Computer perimeter')
        return 2 * math.pi * self.radius

In [16]:
c = Circle(4.0)
c.radius

4.0

In [17]:
c.area

Computer area


50.26548245743669

In [18]:
c.area

50.26548245743669

In [19]:
c.perimeter

Computer perimeter


25.132741228718345

In [20]:
c.perimeter

25.132741228718345